In [1]:
import sklearn
sklearn.__version__
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

import re
from sklearn.metrics import classification_report

In [2]:
data = pd.read_csv("weather_data.csv", delimiter = ";")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8512 entries, 0 to 8511
Data columns (total 12 columns):
time             8512 non-null object
month            8512 non-null int64
temperature      8512 non-null float64
feelslike        8512 non-null float64
wind             8512 non-null float64
direction        8512 non-null object
gust             8512 non-null float64
cloud            8512 non-null float64
humidity         8512 non-null float64
precipitation    8512 non-null float64
pressure         8512 non-null float64
weather          8512 non-null object
dtypes: float64(8), int64(1), object(3)
memory usage: 798.1+ KB


In [3]:
# Xếp loại các loại thời tiết thành 3 dạng là có mưa, mưa ít và không mưa
list_norain = ['Clear', 'Cloudy', 'Mist', 'Sunny', 'Partly cloudy', 'Thundery outbreaks possible']
list_ltrain = ['Light drizzle','Light rain','Light rain shower', 'Patchy light drizzle', 'Patchy light rain', 'Patchy light rain with thunder','Patchy rain possible']
list_rain = ['Heavy rain','Heavy rain at times','Moderate or heavy rain shower','Moderate rain', 'Moderate rain at times', 'Overcast','Torrential rain shower']

In [4]:
# Lấy ra cột y để train
# Cột y này là giá trị thời tiết được xếp loại
y = []
for w in data.weather:
    if w in list_norain:
        y+= ['norain']
    elif w in list_ltrain:
        y+= ['ltrain']
    elif w in list_rain:
        y+= ['rain']
    else:
        y+= ['unknown']
# Do input x có giá trị dự đoán là giá trị thời tiết của 3 giờ tiếp theo
# nên chúng ta xóa dòng đầu tiên
y.pop(0)

'norain'

In [5]:
# Do input x có giá trị dự đoán là giá trị thời tiết của 3 giờ tiếp theo nhưng không có giá trị đó nên xóa dòng cuối cùng
x = data.drop(8511)

In [6]:
# Tách tập train và tập test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, stratify = y, random_state=0)

In [7]:
# Tách tập train và tập validation
x_train_v, x_val, y_train_v, y_val = train_test_split(x_train, y_train, test_size = 0.2, stratify = y_train, random_state=0)

In [8]:
# Góm nhóm các giá trị của các cột có kiểu category
# Cột month gom thành 2 giá trị gồm "mua" (5->11) và "kho" (12->4)
# Cột direction thì xóa bỏ kí tự đầu tiên nếu vượt quá 2 ký tự
# Cột weather thì gom nhóm giống như khi gom nhóm y
class ColStd(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df,y=None):
        df = X_df[['time',
                   'month',
                   'temperature',
                   'feelslike',
                   'wind',
                   'direction',
                   'gust',
                   'cloud',
                   'humidity',
                   'pressure',
                   'weather']]#khong lay cot precipitatinon
        temp_time =[]
        for x in df['time']:
            t = int(re.findall(r'(\d{1,2}):\d{2}',x)[0])
            if ((t>=6) & (t<=15)): 
                temp_time+=['sang']
            else:
                temp_time+=['toi']
        df.time=temp_time
        temp_month =[]
        for x in df['month']:
            if x in [5,6,7,8,9,10,11]:
                temp_month += ['mua'] 
            elif x in [12,1,2,3,4]:
                temp_month += ['kho']
            else:
                temp_month += ['kho']
        df.month = temp_month
        temp_dic = []
        for x in df['direction']:
            if len(x) == 3:
                temp_dic += [x[1:]]
            else:
                temp_dic += [x]
        df.direction = temp_dic
        temp_wea = []
        for w in df['weather']:
            if w in list_norain:
                temp_wea+= ['norain']
            elif w in list_ltrain:
                temp_wea+= ['ltrain']
            elif w in list_rain:
                temp_wea+= ['rain']
            else:
                temp_wea+= ['unknown']
        df.weather = temp_wea
        return df

In [9]:
# Tạo các Pipeline để tiền xử lý các giá trị có trong dữ liệu
nume_cols = ['temperature', 'feelslike', 'wind', 'gust', 'cloud', 'humidity', 'pressure']
#nume_cols_nan = ['temperature', 'feelslike', 'wind', 'gust', 'cloud', 'humidity','precipitation', 'pressure']
cate_cols = ['time', 'month', 'direction', 'weather']

nume_trans = Pipeline(steps=[
    ('imputer1', SimpleImputer(missing_values = np.nan, strategy = 'mean')),
    ('imputer2', SimpleImputer(missing_values = 0, strategy='mean'))
])

cate_trans = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy= 'most_frequent')),
    ('onehot', OneHotEncoder(dtype=int, handle_unknown= 'ignore'))
])

cols_trans = ColumnTransformer(
    transformers=[
        ('nume', nume_trans, nume_cols),
        ('cate', cate_trans, cate_cols)
])

preprocess_pipeline = Pipeline(steps=[
    ('colstd', ColStd()),
    ('coltrans', cols_trans),
    ('std', StandardScaler())
])

# Tiền xử lý tập train x
preprocessed_train_X = preprocess_pipeline.fit_transform(x_train_v)

C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [10]:
# Tiền xử lý tập validation x
preprocess_val_X = preprocess_pipeline.transform(x_val)

C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [11]:
# Tạo ra full_pipeline bao gồm tiền xử lý và model
mlpclf_model = Pipeline(steps=[
    ('pre', preprocess_pipeline),
    ('mlpclf', MLPClassifier(hidden_layer_sizes=(20),activation='tanh', solver='lbfgs', random_state=0, max_iter=500))
])
#hidden_layer_sizes = (10, 20, 50, 100)

In [12]:
# train và tìm ra các tham số tốt nhất
train_errs = []
val_errs = []
iter = []
alphas = [0.001,0.01, 0.1, 1, 10, 100]
hidden_layer_sizes = [1,10,20,50,100]
best_val_err = float('inf'); best_alpha = None; best_hls = None;
for alpha in alphas:
    for hls in hidden_layer_sizes:
        iter +=[f"alpha = {alpha}, hls = {hls}"]
        print(iter[-1])
        mlpclf_model.set_params(mlpclf__alpha=alpha,mlpclf__hidden_layer_sizes=(hls))
        mlpclf_model.fit(x_train_v,y_train_v)
        train_err =100*(1 - mlpclf_model.score(x_train_v,y_train_v))
        val_err =100*(1 - mlpclf_model.score(x_val,y_val))

        train_errs += [train_err]
        val_errs +=[val_err]
        if ((val_err < best_val_err)):
            best_val_err = val_err
            best_alpha = alpha
            best_hls = hls


alpha = 0.001, hls = 1


C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

alpha = 0.001, hls = 10


C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

alpha = 0.001, hls = 20


...

C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

alpha = 100, hls = 50


C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

alpha = 100, hls = 100


C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

In [13]:
for i in range(len(train_errs)):
    print(f"{iter[i]} \n train_err :{train_errs[i]}\t val_err:{val_errs[i]}")
    print()
print(f"best alpha: {best_alpha}")
print(f"best hidden layer size: {best_hls}")

alpha = 0.001, hls = 1 
 train_err :23.567814591153912	 val_err:23.95561357702349

alpha = 0.001, hls = 10 
 train_err :17.08829769871063	 val_err:22.127937336814618

alpha = 0.001, hls = 20 
 train_err :14.542190305206459	 val_err:22.519582245430815

alpha = 0.001, hls = 50 
 train_err :8.633915456177576	 val_err:23.89033942558747

alpha = 0.001, hls = 100 
 train_err :0.03264240248082029	 val_err:27.415143603133163

alpha = 0.01, hls = 1 
 train_err :23.600456993634733	 val_err:24.21671018276762

alpha = 0.01, hls = 10 
 train_err :17.365758119797615	 val_err:21.01827676240209

alpha = 0.01, hls = 20 
 train_err :14.640117512648931	 val_err:21.083550913838124

alpha = 0.01, hls = 50 
 train_err :8.323812632609762	 val_err:24.281984334203656

alpha = 0.01, hls = 100 
 train_err :0.03264240248082029	 val_err:28.00261096605744

alpha = 0.1, hls = 1 
 train_err :23.698384201077193	 val_err:24.086161879895563

alpha = 0.1, hls = 10 
 train_err :17.6268973396442	 val_err:21.148825065274146

In [14]:
svmclf_model = Pipeline(steps=[
    ('pre', preprocess_pipeline),
    ('svm', SVC(kernel = 'rbf', gamma = 0.01 , decision_function_shape='ovr'))
])
c = [0.1,1,10, 20, 50,100]
gamma = [0.0001,0.001,0.01,0.1]
train_errs = []
val_errs = []
iter = []
best_val_err = float('inf'); best_C = None; best_gamma=None
for C in c:
    for g in gamma:
        iter +=[f"C={C}, g={g}"]
        print(iter[-1])
        svmclf_model.set_params(svm__C=C,svm__gamma=g)
        svmclf_model.fit(x_train_v,y_train_v)
        train_err =100*(1 - svmclf_model.score(x_train_v,y_train_v))
        val_err =100*(1 - svmclf_model.score(x_val,y_val))
        train_errs += [train_err]
        val_errs +=[val_err]
        if ((val_err < best_val_err)):
            best_val_err = val_err
            best_C = C
            best_gamma =g
    

C=0.1, g=0.0001


C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

C=0.1, g=0.001


C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

C=0.1, g=0.01


...

C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

C=100, g=0.01


C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

C=100, g=0.1


C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

In [15]:
for i in range(len(train_errs)):
    print(f"{iter[i]} \n train_err :{train_errs[i]}\t val_err:{val_errs[i]}")
    print()
print(f"best C: {best_C}")
print(f"best gamma: {best_gamma}")

C=0.1, g=0.0001 
 train_err :24.579729068059407	 val_err:24.608355091383814

C=0.1, g=0.001 
 train_err :24.579729068059407	 val_err:24.608355091383814

C=0.1, g=0.01 
 train_err :24.579729068059407	 val_err:24.608355091383814

C=0.1, g=0.1 
 train_err :22.523257711767585	 val_err:23.172323759791126

C=1, g=0.0001 
 train_err :24.579729068059407	 val_err:24.608355091383814

C=1, g=0.001 
 train_err :24.579729068059407	 val_err:24.608355091383814

C=1, g=0.01 
 train_err :20.401501550514123	 val_err:21.409921671018274

C=1, g=0.1 
 train_err :16.908764485066097	 val_err:20.822454308093995

C=10, g=0.0001 
 train_err :24.579729068059407	 val_err:24.608355091383814

C=10, g=0.001 
 train_err :24.563407866819	 val_err:24.608355091383814

C=10, g=0.01 
 train_err :18.83466623143464	 val_err:19.712793733681465

C=10, g=0.1 
 train_err :11.702301289374894	 val_err:20.757180156657963

C=20, g=0.0001 
 train_err :24.579729068059407	 val_err:24.608355091383814

C=20, g=0.001 
 train_err :23.6983

In [16]:
#điểm của mô hình mlpclf khi áp dụng tham số tốt nhất dự đoán trên tập tập x_train (có cả tập validation) 
mlpclf_model.set_params(mlpclf__alpha=best_alpha,mlpclf__hidden_layer_sizes=best_hls)
mlpclf_model.fit(x_train, y_train)
y_pred = mlpclf_model.predict(x_train)
print(classification_report(y_train, y_pred))

C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

              precision    recall  f1-score   support

      ltrain       0.70      0.49      0.58      1105
      norain       0.87      0.97      0.92      5776
        rain       0.72      0.43      0.54       778

    accuracy                           0.85      7659
   macro avg       0.76      0.63      0.68      7659
weighted avg       0.83      0.85      0.83      7659



In [17]:
#điểm của mô hình svmclf khi áp dụng tham số tốt nhất dự đoán trên tập tập x_train (có cả tập validation) 
svmclf_model.set_params(svm__C=best_C,svm__gamma=best_gamma)
svmclf_model.fit(x_train,y_train)
y_pred = svmclf_model.predict(x_train)
print(classification_report(y_train, y_pred))

C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


              precision    recall  f1-score   support

      ltrain       0.71      0.35      0.47      1105
      norain       0.84      0.98      0.90      5776
        rain       0.75      0.35      0.47       778

    accuracy                           0.82      7659
   macro avg       0.76      0.56      0.61      7659
weighted avg       0.81      0.82      0.80      7659



# NHẬN XÉT
- mlpclf_model có accuracy cao hơn svmclf_model (0.85>0.82)
- Cái chúng ta quan tâm nhất là khả năng dự đoán được những tình huống mưa và mưa ít
- Ta thấy ở mô hình mlp có precision là thấp hơn so với svm (0.70 < 0.71;0.72 < 0.75), tức là tỉ lệ dự đoán là mưa hoặc ít mưa của svm chính xác hơn
- Tuy nghiên khi xét đến recall, ta thấy mô hình mlp lón hơn đáng kể so với svm(0.49 > 0.35;0.43 > 0.35), tức là mlp model dự đoán được nhiều trường hợp mưa hơn so với svm
- Do nhu cầu chủ yếu là dự đoán được có mưa hay không, tổng kết lại, ta chọn mô hình mlpclf

In [18]:
# Điểm mô hình trên test set
y_pred = mlpclf_model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      ltrain       0.57      0.38      0.46       123
      norain       0.86      0.95      0.90       643
        rain       0.66      0.43      0.52        86

    accuracy                           0.81       852
   macro avg       0.69      0.59      0.63       852
weighted avg       0.79      0.81      0.80       852



C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


# Nhận xét
- Mô hình dự đoán đúng phần lớn trường hợp không mưa
- Trường hợp ít mưa, precision không cao (0.57) và recall rất thấp (0.38)
- Trường hợp mưa, precision có khả quan hơn nhưng còn thấp(0.66) và recall cũng dưới 0.5 (0.43)

# Kết luận
- Mô hình sẽ có hiệu quả không tốt khi áp dụng vào thực tế do chúng ta cần biết có mưa hay không
- Tuy nhiên, theo nhóm mô hình vẫn có giá trị tham khảo khi kết quả dự đoán là mưa (hoặc ít mưa)  